## Collaborative Filtering Using k-Nearest Neighbors (kNN)

Collaborative filtering based on k-nearest neighbors (kNN) kNN is a machine learning algorithm thats finds clusters of similar users based on thier book ratings, and making predictions based on the average of the top (kNN) k-nearest neighbors

K-nearest neighbors (KNN) is a simple, supervised machine learning algorithm that can be used for both classification and regression. This method is easy to implement and understand, but it becomes significantly slow as the size of the data in use increases.

k is a positive integer

N (Nearest)

N (Neighbors)

## Importing Libraries

In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

## Importing Datasets

In [48]:
vamsidhar_books = pd.read_csv('/Users/vamsidharreddy/CMPE-255-Final-Project/data/books_data.csv')
vamsidhar_ratings  = pd.read_csv('/Users/vamsidharreddy/CMPE-255-Final-Project/data/books_ratings_data.csv')
vamsidhar_book_tags = pd.read_csv('/Users/vamsidharreddy/CMPE-255-Final-Project/data/book_tags_data.csv')
vamsidhar_tags = pd.read_csv('/Users/vamsidharreddy/CMPE-255-Final-Project/data/tags_data.csv')

In [68]:
vamsidhar_books

,id,book_id,best_book_id,work_id,books_count,...,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,...,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,...,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,...,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,...,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,...,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...
9994,9995,15613,15613,2764239,199,...,3805,2985,1617,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9995,9996,7130616,7130616,7392860,19,...,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,...,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,...,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...


In [49]:
 #pd.set_option('display.max_columns', 10)

In [50]:
vamsidhar_books.head()

,id,book_id,best_book_id,work_id,books_count,...,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,...,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,...,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,...,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,...,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,...,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [51]:
vamsidhar_books.describe()

,id,book_id,best_book_id,work_id,books_count,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
count,10000.00000,1.000000e+04,1.000000e+04,1.000000e+04,10000.000000,...,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04
mean,5000.50000,5.264697e+06,5.471214e+06,8.646183e+06,75.712700,...,1345.040600,3110.885000,11475.893800,1.996570e+04,2.378981e+04
std,2886.89568,7.575462e+06,7.827330e+06,1.175106e+07,170.470728,...,6635.626263,9717.123578,28546.449183,5.144736e+04,7.976889e+04
min,1.00000,1.000000e+00,1.000000e+00,8.700000e+01,1.000000,...,11.000000,30.000000,323.000000,7.500000e+02,7.540000e+02
25%,2500.75000,4.627575e+04,4.791175e+04,1.008841e+06,23.000000,...,196.000000,656.000000,3112.000000,5.405750e+03,5.334000e+03
50%,5000.50000,3.949655e+05,4.251235e+05,2.719524e+06,40.000000,...,391.000000,1163.000000,4894.000000,8.269500e+03,8.836000e+03
75%,7500.25000,9.382225e+06,9.636112e+06,1.451775e+07,67.000000,...,885.000000,2353.250000,9287.000000,1.602350e+04,1.730450e+04
max,10000.00000,3.328864e+07,3.553423e+07,5.639960e+07,3455.000000,...,456191.000000,436802.000000,793319.000000,1.481305e+06,3.011543e+06


## vamsidhar_ratings info

In [52]:
vamsidhar_ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [53]:
vamsidhar_ratings.describe()

,book_id,user_id,rating
count,981756.000000,981756.000000,981756.000000
mean,4943.275636,25616.759933,3.856534
std,2873.207415,15228.338826,0.983941
min,1.000000,1.000000,1.000000
25%,2457.000000,12372.000000,3.000000
50%,4921.000000,25077.000000,4.000000
75%,7414.000000,38572.000000,5.000000
max,10000.000000,53424.000000,5.000000


### Removing outliers

In [54]:
vamsidhar_books = vamsidhar_books.dropna()
vamsidhar_ratings = vamsidhar_ratings.sort_values("user_id")
vamsidhar_ratings.drop_duplicates(subset=["user_id","book_id"], keep='first', inplace=True) 
vamsidhar_books.drop_duplicates(subset='original_title', keep='first', inplace=True)

###  Using KNN

In [55]:
vamsidhar_merged_df = pd.merge(vamsidhar_books, vamsidhar_ratings, how='left', left_on=['id'], right_on=['book_id'])
vamsidhar_df = vamsidhar_merged_df[['id','original_title', 'user_id', 'rating']]

vamsidhar_df = vamsidhar_df.rename(columns = {'id':'book_id'})
vamsidhar_df.head(200)

,book_id,original_title,user_id,rating
0,1,The Hunger Games,314,5
1,1,The Hunger Games,439,3
2,1,The Hunger Games,588,5
3,1,The Hunger Games,1169,4
4,1,The Hunger Games,1185,4
...,...,...,...,...
195,2,Harry Potter and the Philosopher's Stone,51166,5
196,2,Harry Potter and the Philosopher's Stone,51460,4
197,2,Harry Potter and the Philosopher's Stone,51838,4
198,2,Harry Potter and the Philosopher's Stone,52036,3


In [56]:
vamsidhar_df.describe()

,book_id,user_id,rating
count,763421.000000,763421.000000,763421.000000
mean,4657.948579,25132.161966,3.852257
std,2875.106491,15229.803682,0.980928
min,1.000000,1.000000,1.000000
25%,2123.000000,11843.000000,3.000000
50%,4509.000000,24230.000000,4.000000
75%,7087.000000,38032.000000,5.000000
max,9999.000000,53424.000000,5.000000


In [57]:
vamsidhar_ratings = vamsidhar_df.pivot_table(index='book_id',columns='user_id',values='rating').fillna(0)

#pd.set_option('display.max_columns', 100)
vamsidhar_ratings.head()

user_id,1,2,3,4,5,...,53420,53421,53422,53423,53424
book_id,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0


In [58]:
vamsidhar_ratings.shape

(7755, 50681)

In [59]:
vamsidhar_ratings_matrix = csr_matrix(vamsidhar_ratings.values)

### Training model

In [60]:
vamsidhar_model_knn = NearestNeighbors(metric='cosine', algorithm = 'brute')
vamsidhar_model_knn.fit(vamsidhar_ratings_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

### Some Helper Functions

#### Function which returns book title locations 

In [61]:
def get_book_id(book_title):
    vamsi_target_df = vamsidhar_df.loc[vamsidhar_df['original_title'] == book_title]
    return vamsi_target_df['book_id'].iloc[0]

id_BayouMoon = get_book_id('Bayou Moon')
print(id_BayouMoon)

9996


#### Function which returns book id locations 

In [62]:
def get_title(book_id):
    vamsi_target_df = vamsidhar_df.loc[vamsidhar_df['book_id'] == book_id]
    return vamsi_target_df['original_title'].iloc[0]

print(get_title(1))

The Hunger Games


### This function returns book recommendations using book title

In [63]:
def get_recomm(book_title, num_neighbors=10, display=False): 
    vamsi_book_ids = []
    
    query_index = get_book_id(book_title) - 1
    
    if num_neighbors > 0:
        distances, indices = vamsidhar_model_knn.kneighbors(vamsidhar_ratings.iloc[query_index,:].values.reshape(1, -1), n_neighbors = num_neighbors + 1)
    else:
        distances, indices = vamsidhar_model_knn.kneighbors(vamsidhar_ratings.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 10 + 1)
    
    for i in range(0, len(distances.flatten())):
        if display is True:
            if i == 0:
                print('Recommendations for ', book_title, '\n')
            else:    
                print('{0}\t Book ID: {1}\t  Distance: {2}:\n'.format(i, vamsidhar_ratings.index[indices.flatten()[i]], distances.flatten()[i]))
        
        vamsi_book_ids.append(vamsidhar_ratings.index[indices.flatten()[i]])
    
    return vamsi_book_ids

### Test the Results

### Top 15 recommendations for Harry Potter and the Philosopher's Stone

In [64]:
recommendations_for_HarryPotterandthePhilosophersStone = get_recomm("Harry Potter and the Philosopher's Stone", num_neighbors=15, display=True)

Recommendations for  Harry Potter and the Philosopher's Stone 

1	 Book ID: 4	  Distance: 0.3892592020883805:

2	 Book ID: 33	  Distance: 0.40241308954159594:

3	 Book ID: 13	  Distance: 0.4097614772716248:

4	 Book ID: 5	  Distance: 0.41205601974725015:

5	 Book ID: 19	  Distance: 0.4146445358053812:

6	 Book ID: 28	  Distance: 0.41542064436796167:

7	 Book ID: 18	  Distance: 0.4226021355475864:

8	 Book ID: 7	  Distance: 0.42733371842918766:

9	 Book ID: 15	  Distance: 0.4317700868712365:

10	 Book ID: 43	  Distance: 0.4402457989008367:

11	 Book ID: 1	  Distance: 0.4444738573252147:

12	 Book ID: 24	  Distance: 0.44452856661417617:

13	 Book ID: 23	  Distance: 0.4454417431428892:

14	 Book ID: 10	  Distance: 0.4538274129514055:

15	 Book ID: 14	  Distance: 0.4562715283856553:



In [65]:
for b in recommendations_for_HarryPotterandthePhilosophersStone[1:]:
    print('id:', b, '\t\tBook: ', get_title(b))

id: 4 		Book:  To Kill a Mockingbird
id: 33 		Book:  Memoirs of a Geisha
id: 13 		Book:  Nineteen Eighty-Four
id: 5 		Book:  The Great Gatsby
id: 19 		Book:   The Fellowship of the Ring
id: 28 		Book:  Lord of the Flies 
id: 18 		Book:  Harry Potter and the Prisoner of Azkaban
id: 7 		Book:  The Hobbit or There and Back Again
id: 15 		Book:  Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
id: 43 		Book:  Jane Eyre
id: 1 		Book:  The Hunger Games
id: 24 		Book:  Harry Potter and the Goblet of Fire
id: 23 		Book:  Harry Potter and the Chamber of Secrets
id: 10 		Book:  Pride and Prejudice
id: 14 		Book:  Animal Farm: A Fairy Story


### Top 15 recommendations for To Kill a Mockingbird

In [66]:
book_ids_for_H = get_recomm('To Kill a Mockingbird', num_neighbors=15)
for b in book_ids_for_H[1:]:
    print(get_title(b))

The Great Gatsby
The Catcher in the Rye
Harry Potter and the Philosopher's Stone
Of Mice and Men 
Pride and Prejudice
Animal Farm: A Fairy Story
Memoirs of a Geisha
Nineteen Eighty-Four
Little Women
Lord of the Flies 
Jane Eyre
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
The Lion, the Witch and the Wardrobe
The Scarlet Letter
The Grapes of Wrath


### Top 20 recommendations for The Great Gatsby

In [67]:
book_ids_for_H = get_recomm('The Great Gatsby', num_neighbors=20)
for b in book_ids_for_H[1:]:
    print(get_title(b))

To Kill a Mockingbird
The Catcher in the Rye
Harry Potter and the Philosopher's Stone
Memoirs of a Geisha
Animal Farm: A Fairy Story
Little Women
Of Mice and Men 
The Scarlet Letter
Jane Eyre
Nineteen Eighty-Four
Pride and Prejudice
Lord of the Flies 
The Hobbit or There and Back Again
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
Män som hatar kvinnor
The Lovely Bones
Brave New World
The Grapes of Wrath
An Excellent conceited Tragedie of Romeo and Juliet
 The Fellowship of the Ring


## Converting ipynb to python

In [70]:
!jupyter nbconvert Collaborative*.ipynb --to python

[NbConvertApp] Converting notebook Collaborative_Filtering_vamsidhar.ipynb to python
[NbConvertApp] Writing 5124 bytes to Collaborative_Filtering_vamsidhar.py
